# 🎯 Level 0: The "Aha!" Moment - Credit Scoring

**⏱️ Time**: < 60 seconds  
**🎓 Complexity**: ⭐ Beginner  
**🎯 Goal**: See your first governance audit with minimal code

---

## 📚 What You'll Learn

- How to run a governance policy on raw data
- What a compliance report looks like
- Why data audits matter **before** model training

💡 **No model training needed!** Just data + policy = instant compliance check

This notebook shows how **OSCAL-native governance** (Open Security Controls Assessment Language) makes compliance auditable and version-controlled—just like your code.

### 🎓 Why This Matters (The Real-World Context)

- **Trust**: Transparent audits help stakeholders understand your system's behavior

**Credit scoring models affect people's lives.** A rejected loan application can prevent someone from buying a home, starting a business, or handling an emergency. Historically, ML systems have replicated discriminatory patterns from biased historical data (e.g., redlining, gender discrimination in lending).- **Regulatory compliance**: Many jurisdictions (EU AI Act, US Fair Lending laws) require fairness checks

- **Cheaper to fix**: Data issues are easier to address than retrained models
**Governance-first approach:** By auditing data *before* training, you catch bias early:

## Step 1: Import Libraries and Load Data

First, let's import what we need and load the German Credit dataset.

In [ ]:
from pathlib import Path
import pandas as pd
import venturalitica as vl

print("🚀 Credit Scoring: The Aha! Moment\n")

# Load the German Credit dataset (1,000 real loans)
data_path = Path("data/german_credit.csv")

if not data_path.exists():
    raise FileNotFoundError(
        "❌ Dataset not found. Please run: uv run prepare_data.py\n"
        f"   Expected location: {data_path}"
    )

df = pd.read_csv(data_path)
print(f"✅ Loaded {len(df)} loan applications from local file")

# Basic cleanup to ensure governance columns are present
df['age'] = pd.to_numeric(df.get('age'), errors='coerce')
df = df.dropna(subset=['age'])
df['target'] = pd.to_numeric(df.get('target'), errors='coerce').astype('int64')
if 'gender' not in df.columns:
    df['gender'] = 'unknown'

print(f"   Columns: {', '.join(df.columns[:5])}...")

# Display first few rows for a quick sanity check
df.head()

🚀 Credit Scoring: The Aha! Moment

✅ Loaded 1000 loan applications from local file
   Columns: checking_status, duration, credit_history, purpose, credit_amount...


,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status_sex,other_parties,...,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class,target,gender
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,...,A143,A152,2,A173,1,A192,A201,1,1,male
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,...,A143,A152,1,A173,1,A191,A201,2,0,female
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,...,A143,A152,1,A172,2,A191,A201,1,1,male
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,...,A143,A153,1,A173,2,A191,A201,1,1,male
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,...,A143,A153,2,A173,2,A191,A201,2,0,male


### Why this data matters

**Dataset**: German Credit (UCI ML Repository, 1,000 real loan applications from 1994)
- **Historical context**: One of the earliest fairness benchmarks in ML; frequently used to study discrimination in lending.

- **Protected attributes**: `gender`, `age`, `foreign_worker` status—attributes where discrimination is illegal in most jurisdictions.💡 **Pro tip**: In production, you'd version-control both the dataset *and* the policy file so audits are reproducible.

- **Target**: `target=1` means "creditworthy" (approved loan); `target=0` means "not creditworthy" (rejected).

**Why minimal cleaning?** Real governance audits often happen on raw data to catch issues before expensive transformations. Here we only ensure required columns exist and are numeric so the policy can compute fairness metrics.

**Critical columns for governance**:

- `target`: The decision we're auditing (loan approval)- `age`: Another protected attribute; some regulations require age-group fairness
- `gender`: Protected attribute (male/female); policies check demographic parity here

## Step 2: Point to a Governance Policy

The policy defines what "fair" and "compliant" means for this scenario.

Think of it as "unit tests for fairness"—automated checks that run every time your data or model changes.

### 🎓 What is OSCAL?

**OSCAL (Open Security Controls Assessment Language)** is a NIST standard for machine-readable compliance documentation. Originally designed for cybersecurity, it's perfect for ML governance because:- **Interoperable**: Can be validated by external auditors or regulators

- **Version-controlled**: Policies live in Git alongside your code- **Auditable**: Each control has a unique ID, description, and threshold

In [ ]:
policy_path = Path("policies/loan/data_policy.oscal.yaml")
print(f"✅ Using policy: {policy_path}")
print(f"\n📋 This policy checks:")
print(f"   • Demographic parity (gender fairness)")
print(f"       → Equal approval rates across genders (within tolerance)")
print(f"   • Data quality (class balance)")

print(f"       → Sufficient representation of both classes (approved/rejected)")
print(f"\n💡 Each check is a 'control' with a threshold. Fail = investigate & fix.")

print(f"   • Accuracy calibration")print(f"       → Model predictions should match true positive rates across groups")

✅ Using policy: policies/loan/risks.oscal.yaml
   This policy checks:
   • Demographic parity (gender fairness)
   • Data quality (class balance)
   • Accuracy calibration


## Step 3: Run the Audit 🛡️

This is where the magic happens! No model needed - just data + policy.

**Why this is powerful**: Same `enforce()` call works pre-training (data audit), post-training (model audit), and in production (monitoring). You write the policy once; it runs everywhere.

### 🎓 What `vl.enforce()` Does Under the Hood

4. **Returns structured results**: Pass/fail status, actual values, human-readable messages

1. **Parses the OSCAL policy** to extract control definitions (thresholds, metrics, protected attributes)3. **Evaluates each control**: Compare actual metric values against policy thresholds

2. **Computes fairness metrics** from your data:   - Representation: Sample sizes per protected group

   - Demographic parity: $P(\hat{y}=1 | \text{male}) \approx P(\hat{y}=1 | \text{female})$   - Class balance: $\frac{\text{min}(n_{pos}, n_{neg})}{\text{total}}$

In [ ]:
print("🛡️  Running data quality audit...\n")

# Run the audit - this is the core venturalitica.enforce() call
results = vl.enforce(
    data=df,
    target='target',      # The approval decision column
    gender='gender',      # The protected attribute
    policy=str(policy_path)
)

print("\n✅ Audit complete!")
print(f"📊 Evaluated {len(list(results.values() if isinstance(results, dict) else results))} controls")

🛡️  Running data quality audit...


[Venturalitica] 🛡  Enforcing policy: policies/loan/risks.oscal.yaml
  Evaluating Control 'credit-data-imbalance': Data Quality: Minority class (rejected loans) shou...
    [Binding] Virtual Role 'target' bound to Variable 'target' (Column: 'target')
  Evaluating Control 'credit-data-bias': Pre-training Fairness: Disparate impact ratio shou...
    [Binding] Virtual Role 'target' bound to Variable 'target' (Column: 'target')
    [Binding] Virtual Role 'dimension' bound to Variable 'gender' (Column: 'gender')
  Evaluating Control 'credit-age-disparate': Disparate impact ratio for raw age (Proxy for seni...
    [Binding] Virtual Role 'target' bound to Variable 'target' (Column: 'target')
    [Binding] Virtual Role 'dimension' bound to Variable 'age' (Column: 'age')
  ❌ FAIL | Controls: 2/3 passed
    ✓ [credit-data-imbalance] Data Quality: Minority class (rejected l...: 0.429 (Limit: gt0.2)
    ✓ [credit-data-bias] Pre-training Fairness: Disparate impact

## Step 4: Visualize Results 📊

Let's see what the audit found!

In [ ]:
print("\n" + "="*60)
print("GOVERNANCE AUDIT RESULTS")
print("="*60 + "\n")

raw_results = results.values() if isinstance(results, dict) else results
result_list = list(raw_results)

for r in result_list:
    status = "✅ PASSED" if getattr(r, 'passed', False) else "❌ FAILED"
    control_id = getattr(r, 'control_id', 'unknown_control')
    description = getattr(r, 'description', '')
    message = getattr(r, 'message', '') or getattr(r, 'details', '')
    print(f"{status} | {control_id}")
    if description:
        print(f"   Description: {description}")
    if not getattr(r, 'passed', False) and message:
        print(f"   ⚠️  Issue: {message}")
    actual_value = getattr(r, 'actual_value', None)
    if actual_value is not None:
        print(f"   Value: {actual_value}")
    print()

# Summary
passed = sum(1 for r in result_list if getattr(r, 'passed', False))
total = len(result_list)
print("="*60)
print(f"SUMMARY: {passed}/{total} controls passed")
print("="*60)


GOVERNANCE AUDIT RESULTS

✅ PASSED | credit-data-imbalance
   Description: Data Quality: Minority class (rejected loans) should represent at least 20% of the dataset to avoid biased training due to severe Class Imbalance.
   Value: 0.42857142857142855

✅ PASSED | credit-data-bias
   Description: Pre-training Fairness: Disparate impact ratio should follow the standard '80% Rule' (Four-Fifths Rule), ensuring favorable loan outcomes are representative across groups.
   Value: 0.8965673282047968

❌ FAILED | credit-age-disparate
   Description: Disparate impact ratio for raw age (Proxy for seniority)
   Value: 0.2857142857142857

SUMMARY: 2/3 controls passed


### How to read the report
- **PASSED** controls: data satisfies the policy requirement.
- **FAILED** controls: investigate the message to fix data quality/fairness issues.
- Rerun after fixing data to see compliance improve.

## 🎉 That's it!

You just completed your first governance audit with Venturalitica in less than 60 seconds.

### What happened?
- ✅ The SDK validated your data against a **real OSCAL-native policy**
- ✅ It checked for data quality, fairness, and bias requirements
- ✅ You got actionable results you can show to compliance teams

### Next Steps
Want to see more? Check out:
- `01_governance_audit.ipynb` - Full ML training pipeline with pre/post-training audits
- `02_mlops_integration.py` - Integration with MLflow experiment tracking
- `03_production_ready.py` - Production-grade governance workflows

**Ready to build compliant AI systems? Let's go! 🚀**

### How to read these metrics
- **Status / passed controls**: shows whether each policy control is satisfied. `passed` counts how many controls cleared; `total` is the number evaluated.
- **actual_value**: the measured value for the control (for example, a threshold or rate the policy checks).
- **message / description**: human-readable summary of why a control passed or failed.
- **source_note**: provenance of the data used for the check so you can trace the evidence.

If a control fails, start with its `message` to see which data field or rule triggered the issue, then look at `actual_value` to understand by how much it deviated from the policy expectation.